<center><font size="6"><b>

Часові ряди (Time Series)</b></font></center>

>__Часовим рядом__ називається послідовність значень фактору $y$, який вимірюється через сталі часові інтервали 
$$y_1, ..., y_T, ..., y_t\in \mathbb{R}$$

<img src="https://photos1.blogger.com/blogger/2799/1927/1600/airline1_web.0.jpg" width="600">

###__Компоненти часових рядів__
* __Тренд__ — поступова довгострокова зміна рівня ряду. 
* __Сезонність__ — циклічна зміна рівня ряду зі сталим періодом.
* __Цикл__ — зміна рівня ряду зі змінним періодом (В економіці виділяють цикли довжиною 4−5років, 7−11 років,45−50 років і т. д.
* __Помилка__ — непрогнозована випадкова компонента ряду. До неї входять всі ті характеристики часового ряду, які важко виміряти.


>Працюючи з часовими рядами, необхідно позбутися нестаціонарності часового ряду:
* Сезонності
* Тренду
* Гетероскедастичності



##__Моделі для часових рядів__

Моделі для часових рядів - це прогнозні моделі
$$y_{T+d}\approx f_T(y_1,..., y_T, d),$$
де $d\in\{1,..., D\}$ - зсув прогнозу, $D$ - горизонт прогнозування

Моделі для часових рядів можна поділити на: 

1. Моделі експоненційного згладжування
  * просте експоненці́йне згладжування
  * модель Хольта
  * модель Хольта-Вінтерса
2. Моделі ковзного середнього
  * просте ковзне середнє та авторегресійні моделі
  * ковзне середнє з сезонністю
  * ARMA, ARIMA, SARMA, SARIMA
3. Трендові моделі
  * простий тренд: лінійний, логарифмічний, поліноміальний, експоненційний
  * тренд з сезонністю 
4. Bootstrapping

In [ ]:
#імпортуємо необхідні бібліотеки
%pylab inline
import pandas as pd
from scipy import stats
import statsmodels.api as sm #R-формули
import matplotlib.pyplot as plt
from statsmodels.iolib.table import SimpleTable
from itertools import product


##__Завантажимо датасет__
Щомісячні продажі австралійського вина в тисячах літрів відомі з січня 1980 року по липень 1995 року, необхідно скласти прогноз на найближчі три роки.

In [ ]:
#завантажимо csv-file з локального сховища на вашому диску
from google.colab import files
uploaded = files.upload()

In [ ]:
# перевіримо, чи завантажився файл в каталог
!ls

In [ ]:
# завантажимо датасет з файлу та побудуємо графік часового ряду
wine = pd.read_csv('Monthly-australian-wine-sales.csv',',', index_col=['month'], parse_dates=['month'], dayfirst=True)
plt.figure(figsize(15,7))
wine.sales.plot()
plt.ylabel('Wine sales')
pylab.show()

In [ ]:
wine.head()

##__Автокореляція__
>__Автокореляція__ - це кореляція Пірсона, застосовна до самого ряду (кореляція ряду з самим собою зі зміщенням на лаг).

Автокореляція є однією  основних характеристик часового ряду.



Розглянемо залежність значень об'єму продаж вина в сусідні місяці $y_{t+1}$ та $y_t$

In [ ]:
wine['sales_shift_1']=wine.sales.shift(1)
wine.plot(kind='scatter', x='sales_shift_1', y='sales', figsize=(10, 6), color='darkblue')
plt.xlabel('y_t')
plt.ylabel('y_t+1')
pylab.show()


In [ ]:
wine['sales_shift_2']=wine.sales.shift(2)
wine['sales_shift_3']=wine.sales.shift(3)
wine['sales_shift_S']=wine.sales.shift(12)

fig = plt.figure() 

ac1 = fig.add_subplot(2, 2, 1) 
ac2 = fig.add_subplot(2, 2, 2) 
ac3 = fig.add_subplot(2, 2, 3) 
ac12 = fig.add_subplot(2, 2, 4) 

# 1 місяць
wine.plot(kind='scatter', x='sales_shift_1', y='sales', figsize=(10, 6), color='darkblue',ax=ac1)
ac1.set_title("зв'язок продаж вина в сусідніх місяцях")
ac1.set_xlabel('y_t')
ac1.set_ylabel('y_t+1')

# 2 місяці
wine.plot(kind='scatter', x='sales_shift_2', y='sales', figsize=(10, 6), color='darkblue',ax=ac2)
ac2.set_title("зв'язок продаж вина через місяць")
ac2.set_xlabel('y_t')
ac2.set_ylabel('y_t+2')

# 3 місяці
wine.plot(kind='scatter', x='sales_shift_3', y='sales', figsize=(10, 6), color='darkblue',ax=ac3)
ac3.set_title("зв'язок продаж вина через 2 місяці")
ac3.set_xlabel('y_t')
ac3.set_ylabel('y_t+3')

# 4 місяці
wine.plot(kind='scatter', x='sales_shift_S', y='sales', figsize=(10, 6), color='darkblue',ax=ac12)
ac12.set_title("зв'язок продаж вина через сезон")
ac12.set_xlabel('y_t')
ac12.set_ylabel('y_t+S')

plt.show()


##**Стаціонарність ряду даних**

> Часовий ряд $1, . . . , y_T$ називається __стаціонарим__ якщо $s$ (ширина вікна) розподілу $y_t, . . . , y_{t+s}$ не залежить від $t$, тобто його властивості не залежать від часу.

> Ряди, в яких присутній тренд, нестаціонарні, в залежності від розташування вікна змінюється середній рівень ряду.

> Нестаціонарні ряди можуть мати сезонність: якщо ширина вікна менша за сезонний період, то розподіл ряду буде різним, в залежності від положення вікна.

<img src="http://www.seanabu.com/img/Mean_nonstationary.png" width="600">

>Ряд повинен бути гомоскедастичним, тобто дисперсія має бути сталою, а не залежати від часу

<img src="http://www.seanabu.com/img/Var_nonstationary.png" width="600">

> Коваріація $i$-го  та $(i + m)$-го доданка також не повинна залежати від часу. 

<img src="http://www.seanabu.com/img/Cov_nonstationary.png" width="600">

Можна розглянути два способи перевірки стаціонарності часового ряду. Перший - це перегляд даних. За допомогою візуалізації даних має бути легко визначити змінне середнє значення або варіацію даних. Для більш точної оцінки існує тест Дікі-Фуллера.

##**Тест Дікі-Фуллера**

> У статистиці тест Діккі – Фуллера перевіряє нульову гіпотезу про наявність одиничного кореня в авторегресійній моделі. Альтернативна гіпотеза відрізняється залежно від того, яка версія тесту використовується, але зазвичай означає наявність стаціонарності або тренд-стаціонарності.

Тест перевіряє чи  $\phi=0$ в такій моделі даних:
$$y_t=\alpha+\beta t+\phi y_{t-1}+\epsilon_t$$
яка записується наступним чином
$$Δy_t=y_t-y_{t-1}=\alpha+\beta t+\gamma y_{t-1}+\epsilon_t$$
де  $y_t$  наші дані. Цей запис дає можливість застосувати лінійну регресію до   $Δy_t$ відносно  $t$  та  $y_{t−1}$   та перевірити чи  $γ$ відрізняється від $0$. Якщо $γ=0$, тоді ми маємо довільний процес. Якщо ж ні та  $−1<1+γ<1$, тоді ми маємо стаціонарний ряд.


In [ ]:
test = sm.tsa.adfuller(wine.sales)
print ('adf: ', test[0] )
print ('p-value: ', test[1])
print('Критичні значення: ', test[4])
if test[0]> test[4]['5%']: 
    print ('є одиничний корінь, ряд нестаціонарний')
else:
    print ('відсутній одиничний корінь, ряд стаціонарний')

##__Зведення даних до стаціонарного вигляду__

> __STL-декомпозиція__ (Seasonal Transformation using LOESS - "STL") - 
це метод оцінки нелінійних зв’язків, розбиває часові ряди на компоненти тренду, сезонності та залишків.

In [ ]:
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(wine.sales).plot()
print()

### **Перетворення Бокса-Кокса**

>Перетворення Бокса-Кокса - це спосіб перетворення змінних без нормального розподілу у нормальну форму. Нормальність розподілу є важливим припущенням для багатьох статистичних методів; якщо ваші дані не є нормальнорозподіленими, застосування перетворення Бокса-Кокса означає, що можна в подальшому використовувати більше тестів.

Таке перетворення вводиться для стабілізації дисперсії

В основі перетворення Бокса-Кокса лежить експоненційна змінна, *лямбда* $(\lambda )$, яка змінюється від $-5$ до $5$. Усі значення $\lambda$ враховуються і вибирається оптимальне значення для ваших даних.

"Оптимальне значення" - це те, що дає найкраще наближення нормальної кривої розподілу. Перетворення $y$ має вигляд:
$$y(\lambda)=\left\{\begin{array}{ll}\frac{y^\lambda-1}{\lambda} & \textrm{якщо } \lambda\ne 0\textrm{,}\\ \ln(y) & \textrm{якщо } \lambda=0\textrm{.}\end{array} \right. $$


Тест працює лише для додатніх даних, що зумовлено процесом логарифмування. Якщо вихідні дані мають від'ємні значення, то перетворення має наступний вигляд:
$$y(\lambda)=\left\{\begin{array}{ll}\frac{(y+c)^\lambda-1}{\lambda} & \textrm{якщо } \lambda\ne 0\textrm{,}\\ \ln(y+c) & \textrm{якщо } \lambda=0\textrm{.}\end{array} \right. , \textrm{ де } c - \textrm{ величина зміщення} $$



In [ ]:
wine['sales_box'], lmbda = stats.boxcox(wine.sales)
plt.figure(figsize(15,7))
wine.sales_box.plot()
plt.ylabel(u'Перетворені wine sales')
print("Оптимальний параметр перетворення Бокса-Кокса: %f" % lmbda)
print("Критерій Дікі-Фуллера: p=%f" % sm.tsa.stattools.adfuller(wine.sales_box)[1])

In [ ]:
wine['sales_box_diff'] = wine.sales_box - wine.sales_box.shift(12)
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(wine.sales_box_diff[12:]).plot()
print("Критерій Дікі-Фуллера: p=%f" % sm.tsa.stattools.adfuller(wine.sales_box_diff[12:])[1])

In [ ]:
wine.sales_box_diff[12:]

In [ ]:
wine['sales_box_diff2'] = wine.sales_box_diff - wine.sales_box_diff.shift(1)
plt.figure(figsize(15,10))
sm.tsa.seasonal_decompose(wine.sales_box_diff2[13:]).plot()   
print("Критерій Дікі-Фуллера: p=%f" % sm.tsa.stattools.adfuller(wine.sales_box_diff2[13:])[1])

##__Діаграми кореляції (коррелограмми)__
 
 Зручно аналізувати кореляцію по коррелограммах, де по осі $0x$ відкладаються часовий лаг, а по осі $0y$ - значення автокореляції.

 На кореллограмах є синій коридор навколо осі абсцис - це коридор значимості автокореляції, відмінної від $0$. Значимість стандартно обчислюється за двустороннім критерієм Стьюдента, щоб враховувати і додатню і від'ємну автокореляцію.

**ACF**

>Діаграма автокореляції часового ряду з лагом називається __функцією автокореляції__ або абревіатурою ACF. Цей графік іноді називають _коррелограмою_ або _графіком автокореляції_.

**PACF**

> __Часткова автокореляція__ - це узагальнення взаємозв’язку між спостереженням у часовому ряді зі спостереженнями на попередньому часовому кроці з вилученими зв’язками проміжних спостережень.
 

In [ ]:
plt.figure(figsize(15,8))
ax = plt.subplot(211)
sm.graphics.tsa.plot_acf(wine.sales_box_diff2[13:].values.squeeze(), lags=162, ax=ax)
pylab.show()
ax = plt.subplot(212)
sm.graphics.tsa.plot_pacf(wine.sales_box_diff2[13:].values.squeeze(), lags=48, ax=ax)
pylab.show()

## **Моделювання**

### **Модель ARIMA**

__Модель ARIMA__ (Auto Regressive Integrated Moving Average) - це авторегресійна модель певного порядку, яка комбінується з моделлю ковзного середнього та можливістю диференціювання.

Будь-який "несезонний" часовий ряд, який не є випадковим білим шумом, можна моделювати за допомогою моделей ARIMA.

Модель ARIMA ($p, d, q$) характеризується трьома показниками: $p, d, q$, 

де

$p$ - порядок терміна AR (авторегресія)

$q$ - порядок члена MA (ковзне середнє)

$d$ - необхідна кількість диференціювання для стаціонарності часового ряду

**$AR(p)$**

$$y_t=α+φ_1y_{t−1}+φ_2y_{t−2}+···+φ_py_{t−p}+ε_t$$

 **$MA(q)$**
 
$$y_t=α+ε_t+θ_1ε_{t−1}+θ_2ε_{t−2}+···+θ_qε_{t−q}$$

### **Модель SARIMA**

Якщо у часовому ряді присутня сезонність, необхідно перейти до моделі SARIMA, яка враховує сезонне диференціювання.

Сезонне диференціювання схоже на звичайне диференціювання, але замість віднімання послідовних термінів ви віднімаєте значення з попереднього сезону.

Отже, модель буде представлена у вигляді SARIMA $(p, d, q) \times (P, D, Q)$,

де

$P$ - авторегресійна компонента з урахуванням сезонності ($S$)

$Q$ - компонента ковзного середнього з сезонним кроком $S$

$D$ - порядок сезонного диференціювання 



**$SAR(P)$** $$+φ_Sy_{t−S}+φ_{2S}y_{t−2S}+···+φ_{PS}y_{t−PS}$$

**$SMA(Q)$** $$+θ_Sε_{t−S}+θ_{2S}ε_{t−2S}+···+θ_{PS}ε_{t−QS}$$



In [ ]:
pip install pmdarima


In [ ]:
import pmdarima as pm

###__Початкове наближення__

$Q=1, q=2, P=1, p=4, D=1, d=1$

Початкове наближення можна визначити з коррелограмм та кількості проведених дифернціювань, необхідних для стаціонарності ряду.

Значення $q$ та $Q$ визначаємо з коррелограмми ___ACF___, а $p$ та $P$ значення - з ___PACF___, як номер спостереження останнього значимого коефіцієнта кореляції (за межами синьої зони) для несезонних (з лагом в $1$) та сезонних (з лагом в сезон, наприклад $12$ для року) спостережень, якщо спостерігати від $0$-го спостереження, нехтуючи можливими значимими викидами після незначимих.

Значення $d$ та $D$ визначаються, як порядок несезонного (з лагом в $1$) та сезонного (з лагом в сезон) диференціювання відповідно.



In [ ]:
stepwise_fit = pm.auto_arima(wine.sales_box, start_p=4, start_q=2, max_p=5, max_q=3, m=12,
                             start_P=1,start_Q=1, seasonal=True, d=1, D=1, trace=True,
                             error_action='ignore',  # don't want to know if an order does not work
                             suppress_warnings=True,  # don't want convergence warnings
                             stepwise=True)  # set to stepwise

stepwise_fit.summary()

In [ ]:
 model=sm.tsa.statespace.SARIMAX(wine.sales_box, order=(0, 1, 1), 
                                        seasonal_order=(1, 1, 1, 12)).fit()


**Залишки моделі**

Перевіримо залишки моделі на:

* незміщенність (___критерій Стьюдента___)
* стаціонарність (___критерій Дікі-Фуллера___)
* неавтокорельованість (___коррелограмма___)

In [ ]:
plt.figure(figsize(15,8))
plt.subplot(211)
model.resid[13:].plot()
plt.ylabel(u'Залишки')

ax = plt.subplot(212)
sm.graphics.tsa.plot_acf(model.resid[13:].values.squeeze(), lags=48, ax=ax)

print("T-тест Стьюдента: p=%f" % stats.ttest_1samp(model.resid[13:], 0)[1])
print("Критерій Дікі-Фуллера: p=%f" % sm.tsa.stattools.adfuller(model.resid[13:])[1])

>Зробимо обернене перетворення Бокса-Кокса, щоб повернутися у початкову розмірність 

In [ ]:
def invboxcox(y,lmbda):
   if lmbda == 0:
      return(np.exp(y))
   else:
      return(np.exp(np.log(lmbda*y+1)/lmbda))
wine['model'] =invboxcox(model.fittedvalues, lmbda)
plt.figure(figsize(15,7))
wine.sales.plot()
wine.model[13:].plot(color='r')
plt.ylabel('Wine sales')
pylab.show()

## **Прогноз**

In [ ]:
wine2 = wine[['sales']]
date_list = [datetime.datetime.strptime("1994-09-01", "%Y-%m-%d") + relativedelta(months=x) for x in range(0,36)]
future = pd.DataFrame(index=date_list, columns= wine2.columns)
wine2 = pd.concat([wine2, future])
wine2['forecast'] = invboxcox(model.predict(start=176, end=211), lmbda)

plt.figure(figsize(15,7))
wine2.sales.plot()
wine2.forecast.plot(color='r')
plt.ylabel('Wine sales')
pylab.show()